In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
import logging

try:
    df = pd.read_csv("football.csv", encoding='cp1251')
    
    df['home_team'] = df['home_team'].fillna('unknown')
    df['away_team'] = df['away_team'].fillna('unknown')
    df['weather'] = df['weather'].fillna('clear')
    df['neutral_ground'] = pd.to_numeric(df['neutral_ground'], errors='coerce').fillna(0).astype(int)
    df = df.dropna(subset=['result']) 
    
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    exit()

logging.basicConfig(filename='model_logs.log', level=logging.INFO, format='%(asctime)s - %(message)s')
logging.info("Старт модели...")

print("Распределение классов в данных:")
print(df['result'].value_counts(normalize=True))

X = df[['home_team', 'away_team', 'weather', 'neutral_ground']]
y = df['result']

ros = RandomOverSampler(random_state=42) #балансировка классов, 3 возможных исхода, каждый исход 100:3
X_res, y_res = ros.fit_resample(X, y)

print("\nРаспределение классов после балансировки:")
print(y_res.value_counts(normalize=True))

try:
    X_train, X_test, y_train, y_test = train_test_split(
        X_res, y_res, 
        test_size=0.2, 
        random_state=42,
        stratify=y_res
    )
except ValueError as e:
    print(f"\nОшибка при разделении данных: {e}")
    X_train, X_test, y_train, y_test = train_test_split(
        X_res, y_res, 
        test_size=0.2, 
        random_state=42
    )

cat_features = ['home_team', 'away_team', 'weather']

try:
    model = CatBoostClassifier(
        iterations=50,  #пробовала 50-100 50 наиболее эффективно
        learning_rate=0.05,
        depth=4,
        loss_function='MultiClass',
        cat_features=cat_features,
        verbose=20,
        random_seed=42,
        class_weights=[1, 1, 1],  #равные веса
        early_stopping_rounds=20,
        l2_leaf_reg=3,  #для избежания переобучения
        bootstrap_type='Bernoulli'  #тк датасет <500 
    )
    
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        plot=False
    )
    
    y_pred = model.predict(X_test)
    print(f"\nТочность модели: {accuracy_score(y_test, y_pred):.2f}")
    print("\nОтчет по классификации:")
    print(classification_report(y_test, y_pred))
    
except Exception as e:
    print(f"\nОшибка при обучении модели: {e}")
    exit()

def predict_match():
    print("\nВведите данные:")
    while True:
        home = input("Домашняя команда: ").strip()
        away = input("Гостевая команда: ").strip()
        weather = input("Погода (clear/cloudy/windy): ").strip().lower()
        neutral = input("Нейтральное поле (0-нет, 1-да): ").strip()
        
        if not home or not away:
            print("Ошибка: названия команд не могут быть пустыми")
            continue
            
        if weather not in ['clear', 'cloudy', 'windy']:
            print("Ошибка: погода должна быть clear, cloudy или windy")
            continue
            
        try:
            neutral = int(neutral)
            if neutral not in [0, 1]:
                print("Ошибка: нейтральное поле должно быть 0 или 1")
                continue
        except ValueError:
            print("Ошибка: нейтральное поле должно быть числом 0 или 1")
            continue

        new_data = pd.DataFrame([[home, away, weather, neutral]],
                              columns=['home_team', 'away_team', 'weather', 'neutral_ground'])
        
        prediction = model.predict(new_data)[0]
        proba = model.predict_proba(new_data)[0]
        
        print("\nРезультат:")
        if prediction == 'home_team':
            print(f"Победа {home} (дома) с вероятностью {proba[0]:.1%}")
        elif prediction == 'away_team':
            print(f"Победа {away} (в гостях) с вероятностью {proba[1]:.1%}")
        else:
            print(f"Ничья с вероятностью {proba[2]:.1%}")
        
        print("\nПолные вероятности:")
        print(f"- Победа {home}: {proba[0]:.1%}")
        print(f"- Победа {away}: {proba[1]:.1%}")
        print(f"- Ничья: {proba[2]:.1%}")

        logging.info("Модель завершила работу:)")

        return prediction
            
predict_match()

0:	learn: 1.0933857	test: 1.0961338	best: 1.0961338 (0)	total: 36.4ms	remaining: 5.79s
20:	learn: 1.0234787	test: 1.0528162	best: 1.0528162 (20)	total: 627ms	remaining: 4.15s
40:	learn: 0.9627004	test: 1.0201692	best: 1.0201692 (40)	total: 1.34s	remaining: 3.88s
60:	learn: 0.9037113	test: 0.9965179	best: 0.9965179 (60)	total: 2.06s	remaining: 3.34s
80:	learn: 0.8644152	test: 0.9819039	best: 0.9819039 (80)	total: 2.78s	remaining: 2.71s
100:	learn: 0.8283338	test: 0.9713484	best: 0.9713484 (100)	total: 3.9s	remaining: 2.28s
120:	learn: 0.7959778	test: 0.9657168	best: 0.9649246 (116)	total: 5.18s	remaining: 1.67s
140:	learn: 0.7610690	test: 0.9577109	best: 0.9577109 (140)	total: 6.58s	remaining: 887ms
159:	learn: 0.7321998	test: 0.9525058	best: 0.9525058 (159)	total: 7.73s	remaining: 0us

bestTest = 0.9525058165
bestIteration = 159


Точность модели: 0.53

Введите данные:

Результат:
Победа Пари НН (дома)


array(['home_team'], dtype=object)